In [ ]:
from phsa.adsorption_sites import ClusterAdsorptionSitesFinder
from ase.io import read

filename = ''
atoms = read(filename)
atoms.pbc = False
cas = ClusterAdsorptionSitesFinder(atoms=atoms)
surface_sites = cas.get_surface_sites()
print("surf_sites:",len(surface_sites))
inside_sites = cas.get_inside_sites()
print("inside_sites:",len(inside_sites))
sites = cas.get_sites()
print("sites:",sites)

In [15]:
from ase import Atoms
from phsa.utils import select_points,get_graph,is_unique
from tqdm import trange
num_ad = 2
num_configuration = 3
symbol = 'O'
configurations = []
unique_graphs = []
unique_num = 0
while unique_num < num_configuration:
    structure = atoms.copy()
    points = select_points(cell=atoms.cell,points=sites,num_points= num_ad,min_distance=1.,pbc=atoms.pbc)
    for p in points:
        ad = Atoms(symbols=symbol,positions=[p['position']])
        structure += ad
    graph = get_graph(structure)
    if is_unique(graph,unique_graphs) == True:
        unique_graphs.append(graph)
        configurations.append(structure)
        unique_num += 1